In [24]:
!pip3 uninstall torch
!pip3 install https://download.pytorch.org/whl/cu100/torch-1.0.1.post2-cp36-cp36m-linux_x86_64.whl

Uninstalling torch-1.0.1.post2:
  Would remove:
    /usr/local/bin/convert-caffe2-to-onnx
    /usr/local/bin/convert-onnx-to-caffe2
    /usr/local/lib/python3.6/dist-packages/caffe2/*
    /usr/local/lib/python3.6/dist-packages/torch-1.0.1.post2.dist-info/*
    /usr/local/lib/python3.6/dist-packages/torch/*
Proceed (y/n)? y
  Successfully uninstalled torch-1.0.1.post2
  Using cached https://download.pytorch.org/whl/cu100/torch-1.0.1.post2-cp36-cp36m-linux_x86_64.whl


In [0]:
import torch
from torchtext import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy')
LABEL = data.LabelField(dtype = torch.float)

In [0]:
from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

In [4]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of testing examples: 25000


In [5]:
print(vars(train_data.examples[0]))

{'text': ['This', 'is', 'a', 'strong', 'movie', 'from', 'a', 'historical', 'and', 'epic', 'perspective', '.', 'While', 'the', 'story', 'is', 'simple', 'it', 'is', 'pure', 'and', 'straightforward', '.', 'In', 'truth', ',', 'it', 'is', 'the', 'standard', 'story', 'of', 'a', 'simple', ',', 'honorable', 'man', 'whose', 'honor', 'comes', 'into', 'conflict', 'with', 'the', 'more', 'educated', 'and', 'wealthier', 'men', 'of', 'the', 'period.<br', '/><br', '/>Poor', 'vs.', 'Rich', ',', 'honorable', 'vs.', 'dishonorable', ',', 'a', 'classic', 'but', 'well', '-', 'told', 'tale', 'without', 'much', 'of', 'the', 'glitz', 'of', 'hollywood', 'stinking', 'up', 'the', 'screen.<br', '/><br', '/>Extra', 'points', 'just', 'because', 'you', 'can', 'almost', 'smell', 'the', 'people', 'on', 'the', 'screen', '.', ':)'], 'label': 'pos'}


In [0]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [7]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [0]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [9]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [10]:
print(TEXT.vocab.freqs.most_common(50))

[('the', 201580), (',', 191576), ('.', 164847), ('a', 108961), ('and', 108953), ('of', 100141), ('to', 93048), ('is', 75927), ('in', 60956), ('I', 54460), ('it', 53309), ('that', 48961), ('"', 44121), ("'s", 43211), ('this', 42324), ('-', 37028), ('/><br', 35485), ('was', 35165), ('as', 30314), ('with', 29742), ('movie', 29716), ('for', 29383), ('film', 27056), ('The', 26207), ('but', 24239), ('(', 23199), (')', 23016), ('on', 22916), ("n't", 22901), ('you', 21196), ('are', 20806), ('not', 20243), ('have', 19593), ('his', 19468), ('be', 18522), ('he', 17257), ('one', 16773), ('by', 15189), ('at', 15143), ('!', 15035), ('all', 15011), ('who', 14355), ('an', 14346), ('from', 13665), ('like', 13503), ('they', 13471), ('so', 12355), ('her', 12024), ('or', 11978), ("'", 11811)]


In [11]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'a', 'and', 'of', 'to', 'is']


In [12]:
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7fc8ec1aa2f0>, {'neg': 0, 'pos': 1})


In [0]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

In [0]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):

        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))

In [0]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [16]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,592,105 trainable parameters


In [0]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [0]:
criterion = nn.BCEWithLogitsLoss()

In [0]:
model = model.to(device)
criterion = criterion.to(device)

In [0]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [0]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [24]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 24s
	Train Loss: 0.694 | Train Acc: 50.23%
	 Val. Loss: 0.697 |  Val. Acc: 48.90%
Epoch: 02 | Epoch Time: 0m 24s
	Train Loss: 0.693 | Train Acc: 50.06%
	 Val. Loss: 0.697 |  Val. Acc: 49.03%
Epoch: 03 | Epoch Time: 0m 24s
	Train Loss: 0.693 | Train Acc: 50.29%
	 Val. Loss: 0.698 |  Val. Acc: 50.77%
Epoch: 04 | Epoch Time: 0m 24s
	Train Loss: 0.693 | Train Acc: 49.73%
	 Val. Loss: 0.697 |  Val. Acc: 48.91%
Epoch: 05 | Epoch Time: 0m 25s
	Train Loss: 0.693 | Train Acc: 50.32%
	 Val. Loss: 0.698 |  Val. Acc: 50.72%


In [25]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.710 | Test Acc: 46.64%
